In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [35]:
url = "https://garuda.kemdikbud.go.id/documents?page={}&q=sosiologi"
mainurl = "https://garuda.kemdikbud.go.id"

df = pd.DataFrame.from_dict({
    'name': [],
    'published_at': [],
    'abstract': [],
    'publisher': [],
    'company_publisher': [],
    'creator': [],
    'category': [],
    'web_publisher': [],
    'link_publisher': [],
    # 'ris_citation': [], 
    # 'bibtex_citation': [], 
    # 'ori_source': [], 
    # 'ori_download': [], 
    # 'scholar': [],
})

for i in range(1):
    uri = f"{url.format(i)}"

    result = requests.get(url=uri).content
    doc = BeautifulSoup(result, "html.parser")
    article = doc.findAll('div', class_="article-item")

    for articles in article:
        articletitle = articles.find('a')
        href = articletitle.get('href')
        link_doc = mainurl + href
        spec = requests.get(url=link_doc).content

        journal = BeautifulSoup(spec, "html.parser")
        journal_name = journal.find('h3', class_="ui header").text
        journal_publish = journal.find('div', class_="four wide column").text
        journal_publish_clear = journal_publish.replace("\n", "")
        journal_abstract = journal.find('div', class_="abstract-article").text
        
        segment = journal.find_all("div", class_="ui segment")
        journal_writter_dump = journal.find('div', class_="ui segment article-display")
        journal_writter = journal_writter_dump.find('div')
        if journal_writter:
            middle_div = journal_writter.find('div')
            if middle_div:
                middle_a = middle_div.find('a').text
          
        # journal_writter_fix = journal_writter.find('a').text
        publisher = journal.find("div", class_="ui segment j-art-display")
        name_publisher_meta = publisher.find("div", class_="j-title")
        web_publisher_meta = publisher.find("div", class_="j-website")
        pub_publisher_meta = publisher.find("div", class_="j-pub-name")
        category_publisher_meta = publisher.find("a", class_="ui tag label grey mini").text
        name_publisher = name_publisher_meta.find("a").text
        web_publisher = web_publisher_meta.find("a")
        pub_publisher = pub_publisher_meta.find("a").text
        link_pub_publisher = pub_publisher_meta.find("a")

        web_publisher_fix = web_publisher['href']

        for div in segment:
            item_elements = div.find_all("div", class_="item")
            linked = []
            keys = ['ris_citation', 'bibtex_citation', 'ori_source', 'ori_download', 'scholar']
            for a in item_elements:
                anchor = a.find_all("a", class_="content")
                for index, link in enumerate(anchor):
                    href = link['href']
                    # print(href)
                    linked.append(href)
        
        new_list = [{key:value for key in keys} for value in linked]
        new_row = {'name': journal_name.replace("\n", ""),
            'published_at': journal_publish_clear.replace("Article InfoPublish Date", ""),
            'abstract': journal_abstract.replace("\n", ""),
            'publisher': name_publisher.replace("\n", ""),
            'company_publisher': pub_publisher.replace("\n", ""),
            'category': category_publisher_meta,
            'creator': middle_a,
            'web_publisher': web_publisher_fix.replace("\n", ""),
            'link_publisher': mainurl + link_pub_publisher['href'],
            # 'ris_citation': new_list[0]['ris_citation'],
            # 'bibtex_citation': new_list[0]['bibtex_citation'],
            # 'ori_source': new_list[0]['ori_source'],
            # 'ori_download': new_list[3]['ori_download'],
            # 'scholar': new_list[4]['scholar']
        }
        row_new_df = pd.DataFrame([new_row])
        df = pd.concat([df, row_new_df], ignore_index=True)

In [36]:
df.head(100)

,name,published_at,abstract,publisher,company_publisher,creator,category,web_publisher,link_publisher
0,Gamifikasi Pembelajaran Sosiologi Materi Ragam...,20 May 2020,Abstrak: Desain Pembelajaran Gamifikasi adalah...,JKTP: Jurnal Kajian Teknologi Pendidikan,Universitas Negeri Malang,Luis Rumianda,Other,http://journal2.um.ac.id/index.php/jktp,https://garuda.kemdikbud.go.id/publisher/view/16
1,NINGEN KANKEI TOKOH UTAMA DALAM NOVEL SHINSHO ...,04 Dec 2012,Society consists of several individuals who in...,HUMANIS,Universitas Udayana,Kadek Wahyu Dwi Hartadi,Arts,https://ojs.unud.ac.id/index.php/sastra,https://garuda.kemdikbud.go.id/publisher/view/38
2,ASPEK PERKAWINAN DALAM NOVEL BENANG-BENANG SAM...,22 Dec 2014,"This study, entitled ""Aspek Perkawinan dalam N...",HUMANIS,Universitas Udayana,Ni Kadek Dwijayanti,Arts,https://ojs.unud.ac.id/index.php/sastra,https://garuda.kemdikbud.go.id/publisher/view/38
3,Narasi Sejarah Novel Tambora Karya Agus Sumbog...,01 Feb 2018,The data of this study were taken from an Indo...,HUMANIS,Universitas Udayana,Ardini Mulyadi,Arts,https://ojs.unud.ac.id/index.php/sastra,https://garuda.kemdikbud.go.id/publisher/view/38
4,Analisis Sosiologi Sastra Pada Kumpulan Cerkak...,19 Apr 2017,Abstrak: Penelitian ini bertujuan untuk mendes...,ADITYA - Pendidikan Bahasa dan Sastra Jawa,Universitas Muhammadiyah Purworejo,Faqih Abdurrokhman,Education,http://ejournal.umpwr.ac.id/index.php/aditya,https://garuda.kemdikbud.go.id/publisher/view/25
5,Kajian Sosiologi Cerbung Mandhor Pasar Tandhas...,17 Apr 2017,Abstrak: Penelitian ini bertujuan: (1) mendesk...,ADITYA - Pendidikan Bahasa dan Sastra Jawa,Universitas Muhammadiyah Purworejo,Dona Agnes Pangesti,Education,http://ejournal.umpwr.ac.id/index.php/aditya,https://garuda.kemdikbud.go.id/publisher/view/25
6,Analisis Sosiologi Sastra Dalam Novel Prau Get...,29 Apr 2017,Abstrak.Penelitian ini bertujuan untuk mendesk...,ADITYA - Pendidikan Bahasa dan Sastra Jawa,Universitas Muhammadiyah Purworejo,Sar dono,Education,http://ejournal.umpwr.ac.id/index.php/aditya,https://garuda.kemdikbud.go.id/publisher/view/25
7,Konflik Sosial Dalam Novel Cintrong Paju-Pat K...,22 May 2015,Abstrak: Penelitian ini bertujuan mendeskripsi...,ADITYA - Pendidikan Bahasa dan Sastra Jawa,Universitas Muhammadiyah Purworejo,Emy Ipritania,Education,http://ejournal.umpwr.ac.id/index.php/aditya,https://garuda.kemdikbud.go.id/publisher/view/25
8,Kajian Sosiologi dan Nilai Moralpada Novel Ron...,07 May 2015,Abstrak : Penelitian ini bertujuan untuk (1) m...,ADITYA - Pendidikan Bahasa dan Sastra Jawa,Universitas Muhammadiyah Purworejo,Nur Fitria Anggaeni,Education,http://ejournal.umpwr.ac.id/index.php/aditya,https://garuda.kemdikbud.go.id/publisher/view/25
9,ANALISIS SOSIOLOGI SASTRA NOVEL PEREMPUAN KARY...,29 Sep 2018,Abstrak: Penelitian ini bertujuan untuk mendes...,Jurnal Bahtera - Jurnal Pendidikan Bahasa dan ...,Universitas Muhammadiyah Purworejo,Nursitatun Nursitatun,Education,http://ejournal.umpwr.ac.id/index.php/surya-ba...,https://garuda.kemdikbud.go.id/publisher/view/25


In [ ]:
df.to_csv('garuda_grc.csv', index=False)

In [20]:
df.to_excel('garuda_grc.xlsx', index=False)